# Introdução

## Importação das bibliotecas necessarias

In [2]:
import requests
import pandas as pd
import os
import json
import sys
import matplotlib.pyplot as plt
import funcoes_comuns as fc
from importlib import reload
reload(fc)

<module 'funcoes_comuns' from '/home/pcdas/gabriel.silva.w1n3/team-vmmm/near_miss_ufrj/calculos/blocagem/funcoes_comuns.py'>

# Dataframe base de Código do município e Ano

In [3]:
base_df = fc.get_tb_municipios()
base_df.head()

,CODIGO_MUNICIPIO,ano_nasc,CODMUNRES
0,1100015,2012,110001
1,1100015,2013,110001
2,1100015,2014,110001
3,1100015,2015,110001
4,1100015,2016,110001


# Nascidos vivos de mulheres de 2012 a 2020

In [4]:
%time
df_t_nasc_vivos = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as TOTAL_DE_NASCIDOS_VIVOS \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2020 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 26.5 µs
...................................................(y)
Record rows length : 50268


# Quantidade de baixo peso ao nascer

In [5]:
%time

df_baixo_peso = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as NASCIDOS_VIVOS_COM_BAIXO_PESO \
                    FROM \"datasus-sinasc\" \
                    WHERE  ano_nasc>=2012 AND \
                    ano_nasc<=2020 AND \
                    PESO<2500 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 0 ns
..................................................(y)
Record rows length : 49039


# Quantidade de nascimentos prematuros

In [6]:
%time

df_nascimentos_prematuros = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as NASCIDOS_VIVOS_PREMATUROS \
                    FROM \"datasus-sinasc\" \
                    WHERE  ano_nasc>=2012 AND \
                    ano_nasc<=2020 AND \
                    GESTACAO<5 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.87 µs
..................................................(y)
Record rows length : 49557


# Número de nascimentos termo precoce

In [7]:
%time

df_termo_precoce = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as NASCIDOS_VIVOS_TERMO_PRECOCE \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2020 AND \
                    SEMAGESTAC>=37 AND \
                    SEMAGESTAC<=38 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 0 ns
...................................................(y)
Record rows length : 50176


# Finalização

In [11]:
# Conversão do CODMUNRES para inteiro, para que seja possível a junção
base_df['CODMUNRES']=base_df['CODMUNRES'].astype(int)
df_t_nasc_vivos['CODMUNRES']=df_t_nasc_vivos['CODMUNRES'].astype(int)
df_baixo_peso['CODMUNRES']=df_baixo_peso['CODMUNRES'].astype(int)
df_nascimentos_prematuros['CODMUNRES']=df_nascimentos_prematuros['CODMUNRES'].astype(int)
df_termo_precoce['CODMUNRES']=df_termo_precoce['CODMUNRES'].astype(int)

# Junção em um dataframe final
df_final = pd.merge(base_df, df_t_nasc_vivos, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_baixo_peso, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_nascimentos_prematuros, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_termo_precoce, how='left', on=['CODMUNRES','ano_nasc'])

# Preencher NaN com zeros
df_final = df_final.fillna(0)
df_final.rename(columns={'ano_nasc': 'ANO'},
               inplace=True)

del df_final['CODIGO_MUNICIPIO']

aux = [i for i in range(len(df_final.columns.values.tolist()))]
aux[0] = 1
aux[1] = 0
df_final = df_final.iloc[:, aux]

In [12]:
df_final.head()

,CODMUNRES,ANO,TOTAL_DE_NASCIDOS_VIVOS,NASCIDOS_VIVOS_COM_BAIXO_PESO,NASCIDOS_VIVOS_PREMATUROS,NASCIDOS_VIVOS_TERMO_PRECOCE
0,110001,2012,386.0,11.0,29.0,106.0
1,110001,2013,382.0,15.0,23.0,106.0
2,110001,2014,341.0,28.0,28.0,89.0
3,110001,2015,373.0,22.0,36.0,105.0
4,110001,2016,326.0,22.0,35.0,96.0


In [13]:
df_final.columns.values.tolist()

['CODMUNRES',
 'ANO',
 'TOTAL_DE_NASCIDOS_VIVOS',
 'NASCIDOS_VIVOS_COM_BAIXO_PESO',
 'NASCIDOS_VIVOS_PREMATUROS',
 'NASCIDOS_VIVOS_TERMO_PRECOCE']

In [14]:
df_final.to_csv('./indicadores_bloco5_condicao_de_nascimento_2012-2020.csv',index=False)